<a href="https://colab.research.google.com/github/sagihaider/MEG_GAN_2021/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
from keras.layers import Input, Dense, Activation, BatchNormalization, PReLU, Dropout
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt




def get_optimizer():
    #return SGD(lr =  0.01, momentum = 0.1)
    return Adam(lr = 0.0003)

def build_models(n_neurons):
    """Creates three different models, one used for source only training, two used for domain adaptation"""
    inputs = Input(shape=(12,)) # We have 20 variables in the EEG data from FBCSP
    x4 = Dense(32, activation='linear', kernel_regularizer = "l2")(inputs)
    x4 = BatchNormalization()(x4)
    x4 = Activation("tanh")(x4)  
    x4 = Dropout(0.5)(x4)
    
    x4 = Dense(2, activation='linear', kernel_regularizer = "l2")(x4)
    x4 = BatchNormalization()(x4)
    x4 = Activation("tanh")(x4)  

    source_classifier = Dense(2, activation='softmax', name="mo",  kernel_regularizer = "l2")(x4)  
    
    domain_classifier = Dense(32, activation='linear', name="do4", kernel_regularizer = "l2", use_bias = False)(x4)
    domain_classifier = BatchNormalization(name="do5")(domain_classifier)
    domain_classifier = Activation("elu", name="do6")(domain_classifier)
    #domain_classifier = Dropout(0.25)(domain_classifier)

    domain_classifier = Dense(2, activation='softmax',  kernel_regularizer = "l2", name="do")(domain_classifier)

    comb_model = Model(inputs=inputs, outputs=[source_classifier, domain_classifier])
    comb_model.compile(optimizer=get_optimizer(),
              loss={'mo': 'categorical_crossentropy', 'do': 'categorical_crossentropy'},
              loss_weights={'mo': 1, 'do': 1}, metrics=['accuracy'], )

    source_classification_model = Model(inputs=inputs, outputs=[source_classifier])
    source_classification_model.compile(optimizer=get_optimizer(),
              loss={'mo': 'categorical_crossentropy'}, metrics=['accuracy'], )


    domain_classification_model = Model(inputs=inputs, outputs=[domain_classifier])
    domain_classification_model.compile(optimizer=get_optimizer(),
                  loss={'do': 'categorical_crossentropy'}, metrics=['accuracy'])
    
    
    embeddings_model = Model(inputs=inputs, outputs=[x4])
    embeddings_model.compile(optimizer=get_optimizer(),loss = 'categorical_crossentropy', metrics=['accuracy'])
                        
    comb_model.summary()      
    source_classification_model.summary()          
    domain_classification_model.summary()       
    embeddings_model.summary()
    return comb_model, source_classification_model, domain_classification_model, embeddings_model

def batch_generator(data, batch_size):
    """Generate batches of data.

    Given a list of numpy data, it iterates over the list and returns batches of the same size
    This
    """
    all_examples_indices = len(data[0])
    while True:
        mini_batch_indices = np.random.choice(all_examples_indices, size=batch_size, replace=False)
        tbr = [k[mini_batch_indices] for k in data]
        yield tbr


# In[ ]:


#SAMPLING_ITERATIONS = 3000

def train(Xs, ys, Xt, yt,  enable_dann = True, n_iterations = 1000):
    
    batch_size = len(Xs)
    
    model, source_classification_model, domain_classification_model, embeddings_model = build_models(2) # Number of neurons for shared layer

    y_class_dummy = np.ones((len(Xt), 2))
    y_adversarial_1 = to_categorical(np.array(([1] * batch_size + [0] * batch_size)))
    
    sample_weights_class = np.array(([1] * batch_size + [0] * batch_size))
    sample_weights_adversarial = np.ones((batch_size * 2,))

    S_batches = batch_generator([Xs, to_categorical(ys)], batch_size)
    T_batches = batch_generator([Xt, np.zeros(shape = (len(Xt),2))], batch_size)
    
    for i in range(n_iterations):
        # # print(y_class_dummy.shape, ys.shape)
        y_adversarial_2 = to_categorical(np.array(([0] * batch_size + [1] * batch_size)))

        X0, y0 = next(S_batches)
        X1, y1 = next(T_batches)


        X_adv = np.concatenate([X0, X1])
        y_class = np.concatenate([y0, np.zeros_like(y0)])

        adv_weights = []
        for layer in model.layers:
            if (layer.name.startswith("do")):
                adv_weights.append(layer.get_weights())

        if(enable_dann):
            # note - even though we save and append weights, the batchnorms moving means and variances
            # are not saved throught this mechanism 
            stats = model.train_on_batch(X_adv, [y_class, y_adversarial_1],
                                     sample_weight=[sample_weights_class, sample_weights_adversarial])
            
            k = 0
            for layer in model.layers:
                if (layer.name.startswith("do")):
                    layer.set_weights(adv_weights[k])
                    k += 1

            class_weights = []
            
        
            for layer in model.layers:
                if (not layer.name.startswith("do")):
                    class_weights.append(layer.get_weights())
            
            stats2 = domain_classification_model.train_on_batch(X_adv, [y_adversarial_2])

            k = 0
            for layer in model.layers:
                if (not layer.name.startswith("do")):
                    layer.set_weights(class_weights[k])
                    k += 1

        else:
            source_classification_model.train_on_batch(X0,y0)
            
       
        # if ((i + 1) % 1000 == 0):
        #     # print(i, stats)
        #     y_test_hat_t = source_classification_model.predict(Xt).argmax(1)
        #     y_test_hat_s = source_classification_model.predict(Xs).argmax(1)
        #     print("Iteration %d, source accuracy =  %.3f, target accuracy = %.3f"%(i, accuracy_score(ys, y_test_hat_s), accuracy_score(yt, y_test_hat_t)))
        #

    y_test_hat_t = source_classification_model.predict(Xt).argmax(1)
    y_test_hat_s = source_classification_model.predict(Xs).argmax(1)
    tr_acc=accuracy_score(ys, y_test_hat_s)
    ts_acc=accuracy_score(yt, y_test_hat_t)
    #print("Training Acc at 1500 itr: %.3f"%(tr_acc))

            
    return embeddings_model, source_classification_model, tr_acc, ts_acc



def bootstrap_train(Xs, ys, Xt, yt, enable_dann,  n_iterations = 100, n_bootstrap_iterations = 50):

    y_test_hat_t_b = []
    y_test_hat_s_b = []

    for j in range(n_bootstrap_iterations):
        all_examples_indices = len(Xs)
        mini_batch_indices = np.random.choice(
            all_examples_indices,
            size=all_examples_indices,
            replace=True
        )

        embs, model,  tr_acc, ts_acc = train(Xs[mini_batch_indices], ys[mini_batch_indices], Xt, yt, enable_dann=enable_dann, n_iterations=n_iterations)
        print(j, "tr_acc", tr_acc, "tr_acc", ts_acc)
        y_test_hat_t = model.predict(Xt)
        y_test_hat_s = model.predict(Xs)
        y_test_hat_t_b.append(y_test_hat_t)
        y_test_hat_s_b.append(y_test_hat_s)

    y_test_hat_t_b = np.array(y_test_hat_t_b)
    y_test_hat_s_b = np.array(y_test_hat_s_b)

    print(y_test_hat_t_b.shape)

    tr_acc = accuracy_score(ys, y_test_hat_s_b.mean(axis = 0).argmax(1))
    ts_acc = accuracy_score(yt, y_test_hat_t_b.mean(axis = 0).argmax(1))

    return embs, model, tr_acc, ts_acc


In [16]:
import scipy.io as spio
from numpy import zeros

cols = 2
rows = 2
results = zeros([rows, cols])
results_dann = zeros([rows, cols])

In [19]:
for x in (range(1, 2)):
    fName = '/content/P00'+ str(x) + 'ClassComb_1_2_Features.mat'  # Load Data
    print(fName)
    mat = spio.loadmat(fName)
    Xs = mat['features_tr']
    Xs1 = Xs[0,0]
    Xs2 = Xs[0,1]
    Xs = np.concatenate((Xs1, Xs2), axis=1)
    ys = mat['Y_tr']
    yt = mat['Y_ts']
    Xt = mat['features_ts']
    Xt1 = Xt[0,0]
    Xt2 = Xt[0,1]
    Xt = np.concatenate((Xt1, Xt2), axis=1)

    embs, source, tr_acc, ts_acc = bootstrap_train(Xs, ys, Xt, yt, enable_dann = False, n_iterations = 2000)
    print(tr_acc, ts_acc)
    results[x-1,0]=tr_acc
    results[x-1,1]=ts_acc
    
    # Train a Learning Model with Domain Adaptation
    embs, tr_acc, ts_acc = bootstrap_train(Xs, ys, Xt, yt, enable_dann = True, n_iterations = 15000)
    results_dann[x-1,0]=tr_acc
    results_dann[x-1,1]=ts_acc

print(results)
print(results_dann)

/content/P001ClassComb_1_2_Features.mat
Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 12)]         0                                            
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 32)           416         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 32)           128         dense_8[0][0]                    
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 32)           0           batch_normalization_8[0][0]      
___________________________________________________

ValueError: ignored